## Background
Simulating quantum systems is one of the most important potential applications of quantum computers. Its main challenge is to find an efficient circuit that asymptotically approximates the time evolution of a Hamiltonian. The product formula approach has a straightforward implementation and good performance in practice. Given a Hamiltonian that is decomposed as a sum of Hermitian terms, the product formula approximates the exponential of this Hamiltonian as a product of exponentials of individual terms and each exponential can be efficiently realized by a quantum circuit. 

This high-level circuit representation is typically hardware agnostic and needs to be decomposed into the instruction set supported by the underlying quantum hardware. In noisy intermediatescale quantum (NISQ) computers, the universal instruction set is normally composed of arbitrary single-qubit rotations plus one or a few two-qubit gates (e.g., the SYC, CNOT, iSWAP gates from Google, IBM, Rigetti respectively ). Furthermore, these quantum computers only allow two-qubit gates between specific qubit pairs, i.e., there is limited qubit connectivity. Compilation techniques are required to map circuit qubits onto hardware qubits and insert SWAP gates to move qubits into neighbouring positions, increasing circuit size in terms of gate count and circuit depth. Two-qubit gates have much higher error rates than single-qubit rotations and qubits have short coherence time. Therefore, it is critical to minimize compilation overhead for high-fidelity circuit implementation.

2QAN identifies the flexibility in the Hamiltonian operator permutation and exploit it in the compilation procedure. In particular, it consists of permutation-aware qubit routing, gate scheduling, and gate optimization techniques to efficiently compile circuits for 2-local qubit Hamiltonian simulation problems. For more details, check out our [2QAN paper](https://arxiv.org/abs/2108.02099). In the following examples, we will walk through these compilation passes.

In [1]:
# Import 2QAN compiler passes
 
# BenchArch defines the qubit architecture used in 2QAN
# and translates an OpenQASM circuit into Qiskit circuit.
from py2qan import BenchArch

# HeuristicMapper is the qubit mapping pass, 
# which finds an efficient mapping between circuit qubits and device qubits
# to reduce #SWAPs during qubit routing
from py2qan import HeuristicMapper

# QuRouter consists of a permutation-aware qubit routing pass 
# and a permutation-aware gate scheduling pass to minimise #SWAPs and circuit depth
from py2qan import QuRouter

## Benchmark circuit
The example circuit is a one-layer QAOA circuit for solving max-cut problems.

In [2]:
import os
import pickle as pkl
# Import qiskit 
import qiskit
from qiskit import transpile, QuantumCircuit

# Benchmarks qaoa is True only if the inputs are QAOA benchmarks
qaoa = True 
# For QAOA benchmarks, the parameters differ in different layers. 
# For other quantum simulation circuits, 
# we use the standard product formula implementation, i.e., the parameters are the same in each layer

# OpenQASM circuits here only contain one layer/depth
with open(os.path.join('qaoa_qasms.pkl'), 'rb') as f:
    qasms = pkl.load(f)
# The parameters here include gammas for rzz and betas for rx in 4 layers
with open(os.path.join('qaoa_params.pkl'), 'rb') as f:
    params = pkl.load(f)
idx = -2  # circuit id
c_qasm = qasms[idx]

param = None
if qaoa:
    param = params[idx]

## Device restrictions
2QAN considers the following device restrictions: native gate set, qubit connectivity (architecture). For example,

In [3]:
import numpy as np

# Device gate set, assume cx(cnot) as the native two-qubit gate
basis_gates = ['id', 'rz', 'u3', 'u2', 'cx', 'reset']

# Device topology, assume a grid architecture as an example
qn = 20 # The number of qubits on the fake device
lattice_xy=(4,5)
    
# Define the 2QAN qubit architecture, given the grid size lattice_xy and the inpout circuit c_qasm
grid_topology = BenchArch(c_qasm, lattice_xy=lattice_xy).topology
# Generate the coupling map/qubit connectivity
coupling_map = [list(edge) for edge in list(grid_topology.edges)]
coupling_map += [[edge[1], edge[0]] for edge in list(grid_topology.edges)]

## Map circuit qubits to device qubits: initial qubit mapping
One can use the default mapper ['qap'](https://github.com/lllingoo/2QAN/blob/master/py2qan/heuristic_mapper.py) for small circuits or 'qiskit' [SABRE](https://qiskit.org/documentation/stubs/qiskit.transpiler.passes.SabreSwap.html) mapper for large circuits. Both QAP and Qiskit SABRE mappers output inital qubit maps randomly, one can run the mapper several times to achieve better compilation results

In [4]:
mapper = 'qiskit' 
hmapper = HeuristicMapper(c_qasm, coupling_map=coupling_map) # The mapping pass in 2QAN
if mapper == 'qap':
    # The default mapper based on Quadratic Assignment Problem, very slow for #qubits > 30
    init_map, cost = hmapper.run_qap(num_iter=200, lst_len=20)
elif mapper == 'qiskit':
    # The SABRE mapper in Qiskit
    init_map = hmapper.run_qiskit(max_iterations=5)
    
print('The initial qubit map is \n', init_map) # keys are device qubits, values are circuit qubits

The initial qubit map is 
 {0: 0, 1: 13, 2: 15, 3: 6, 4: 9, 5: 3, 6: 17, 7: 16, 8: 5, 9: 12, 10: 19, 11: 10, 12: 4, 13: 8, 14: 7, 15: 2, 16: 18, 17: 1, 18: 14, 19: 11}


## Routing and scheduling
2QAN performs permutation-aware routing to minimise #SWAPs and permutation-aware scheduling to minimise circuit depth. The routing and scheduling passes are packed in [QuRouter](https://github.com/lllingoo/2QAN/blob/master/py2qan/qrouter_depth.py). It does not perform gate decompostion before routing and scheduling. During routing, 2QAN merges a SWAP gate and a circuit two-qubit gate into one two-qubit gate if they act on the same qubit pair.

In [5]:
layers = 1 # The number of QAOA layers
# Routing and scheduling, takes init_map as input
router = QuRouter(c_qasm, init_map=init_map, coupling_map=coupling_map) # The routing and scheduling passes in 2QAN
if qaoa:
    # For QAOA, different layers have different gate parameters
    qs_circ, swaps1 = router.run_qaoa(layers=layers, gammas=param[layers-1][:layers], betas=param[layers-1][layers:], msmt=True) 
else:
    # For quantum simulation circuits, we assume each layer has the same time steps/parameters
    qs_circ, swaps1 = router.run(layers=layers, msmt='True')
    
# The routed and scheduled circuit: 
# all two-qubit gates are nearest-neighbouring gates on the device, but they have not been decompsed into native gate set
qs_circ.draw()

┌───┐                                                                  »
 q_0: ┤ H ├─■────────────────────────────────────────────────────────────────»
      ├───┤ │ZZ(0.60866)                                                     »
 q_1: ┤ H ├─■───────────────────■────────────────────────────────────────────»
      ├───┤                     │ZZ(0.60866)                                 »
 q_2: ┤ H ├─────────────────────■────────────────────────────────────────────»
      ├───┤              ┌────────────────────────┐                          »
 q_3: ┤ H ├──────────────┤0                       ├──────────────────────────»
      ├───┤              │                        │┌────────────────────────┐»
 q_4: ┤ H ├─■────────────┤                        ├┤0                       ├»
      ├───┤ │ZZ(0.60866) │                        ││                        │»
 q_5: ┤ H ├─■────────────┤  dZZ0.6086626576954088 ├┤                        ├»
      ├───┤              │                        ││                        │»
 q_6: ┤ H ├──────────────┤                        ├┤  dZZ0.6086626576954088 ├»
      ├───┤              │                        ││                        │»
 q_7: ┤ H ├─■────────────┤1                       ├┤                        ├»
      ├───┤ │            └────────────────────────┘│                        │»
 q_8: ┤ H ├─┼───────────────────■──────────────────┤1                       ├»
      ├───┤ │                   │                  ├────────────────────────┤»
 q_9: ┤ H ├─┼───────────────────┼──────────────────┤0                       ├»
      ├───┤ │                   │                  │                        │»
q_10: ┤ H ├─┼───────────────────┼──────────────────┤                        ├»
      ├───┤ │ZZ(0.60866)        │                  │                        │»
q_11: ┤ H ├─■───────────────────┼──────────────────┤  dZZ0.6086626576954088 ├»
      ├───┤                     │ZZ(0.60866)       │                        │»
q_12: ┤ H ├─────────────────────■──────────────────┤                        ├»
      ├───┤                                        │                        │»
q_13: ┤ H ├──────────────────────────X─────────────┤1                       ├»
      ├───┤                          │             └────────────────────────┘»
q_14: ┤ H ├─■────────────────────────X────────────────────■──────────────────»
      ├───┤ │                                             │ZZ(0.60866)       »
q_15: ┤ H ├─┼─────────────────────────────────────────────■──────────────────»
      ├───┤ │                                                                »
q_16: ┤ H ├─┼────────────────────────────────────────────────────────────────»
      ├───┤ │                                                                »
q_17: ┤ H ├─┼────────────────────────────────────────────────────────────────»
      ├───┤ │ZZ(0.60866)                                                     »
q_18: ┤ H ├─■───────────────────■────────────────────────────────────────────»
      ├───┤                     │ZZ(0.60866)                                 »
q_19: ┤ H ├─────────────────────■────────────────────────────────────────────»
      └───┘                                                                  »
c: 20/═══════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                            »
« q_0: ──────────────────────────────────────────────────────»
«                                                            »
« q_1: ──────────────────────────────────────────────X───────»
«                                                    │       »
« q_2: ─■───────────────────■────────────────────────┼───────»
«       │                   │ZZ(0.60866)             │       »
« q_3: ─┼───────────────────■────────────────────────┼───────»
«       │                                            │       »
« q_4: ─┼─────────────────────────────────

## Decompostion and optimisation
2QAN does not perform gate decompostion and optimisation, one can use different open-sourced compilers for decomposing circuit into different native two-qubit gates. 
1. For the cx/cnot gate set, we can use [Qiskit](https://github.com/Qiskit) compiler to perform decompostion and optimisation. 
2. For QAOA and Ising models, we can use [Cirq](https://github.com/quantumlib/Cirq) compiler to decompose them into SYC gates. 
3. For other applications, we can use the numerical decompostion approach [NuOp](https://github.com/prakashmurali/NuOp).

In [6]:
# Assuming the cx/cnot gate set in IBMQ
decom_circ = transpile(qs_circ, basis_gates=basis_gates, optimization_level=3)
decom_circ.draw()

global phase: 5.733
        ┌─────────┐                                                          »
 q_0: ──┤ U2(0,π) ├─────■───────────────────────────────────────■────────────»
        ├─────────┤   ┌─┴─┐┌───────────────────────┐          ┌─┴─┐          »
 q_1: ──┤ U2(0,π) ├───┤ X ├┤ Rz(0.608662657695409) ├──────────┤ X ├──────────»
        ├─────────┤   └───┘└───────────────────────┘          └───┘          »
 q_2: ──┤ U2(0,π) ├──────────────────────────────────────────────────────────»
      ┌─┴─────────┴──┐                                                       »
 q_3: ┤ U3(π,-π/2,0) ├───────────────────────────────────────────────────────»
      └─┬─────────┬──┘                                                       »
 q_4: ──┤ U2(0,π) ├─────■───────────────────────────────────────■────────────»
        ├─────────┤   ┌─┴─┐┌───────────────────────┐          ┌─┴─┐          »
 q_5: ──┤ U2(0,π) ├───┤ X ├┤ Rz(0.608662657695409) ├──────────┤ X ├──────────»
        ├─────────┤   └───┘└───────────────────────┘          └───┘          »
 q_6: ──┤ U2(0,π) ├──────────────────────────────────────────────────────────»
        ├─────────┤                                                          »
 q_7: ──┤ U2(0,π) ├─────■────────────────────────────────────────────────────»
        ├─────────┤     │                                                    »
 q_8: ──┤ U2(0,π) ├─────┼──────────────■─────────────────────────────────────»
      ┌─┴─────────┴──┐  │              │                                     »
 q_9: ┤ U3(π,-π/2,0) ├──┼──────────────┼─────────────────────────────────────»
      ├──────────────┤  │              │                                     »
q_10: ┤ U3(π,-π/2,0) ├──┼──────────────┼─────────────────────────────────────»
      └─┬─────────┬──┘┌─┴─┐            │            ┌───────────────────────┐»
q_11: ──┤ U2(0,π) ├───┤ X ├────────────┼────────────┤ Rz(0.608662657695409) ├»
        ├─────────┤   └───┘          ┌─┴─┐          ├───────────────────────┤»
q_12: ──┤ U2(0,π) ├──────────────────┤ X ├──────────┤ Rz(0.608662657695409) ├»
        ├─────────┤                  └───┘          └───────────────────────┘»
q_13: ──┤ U2(0,π) ├──────────────────────────────────────────────────────────»
        ├─────────┤                                                          »
q_14: ──┤ U2(0,π) ├─────■───────────────────────────────────────■────────────»
        ├─────────┤     │                                       │            »
q_15: ──┤ U2(0,π) ├─────┼───────────────────────────────────────┼────────────»
        ├─────────┤     │                                       │            »
q_16: ──┤ U2(0,π) ├─────┼───────────────────────────────────────┼────────────»
        ├─────────┤     │                                       │            »
q_17: ──┤ U2(0,π) ├─────┼───────────────────────────────────────┼────────────»
        ├─────────┤   ┌─┴─┐┌───────────────────────┐          ┌─┴─┐          »
q_18: ──┤ U2(0,π) ├───┤ X ├┤ Rz(0.608662657695409) ├──────────┤ X ├──────────»
        ├─────────┤   └───┘└───────────────────────┘          └───┘          »
q_19: ──┤ U2(0,π) ├──────────────────────────────────────────────────────────»
        └─────────┘                                                          »
c: 20/═══════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                                     »
« q_0: ───────────────────────────────────────────────────────────────»
«                                                                     »
« q_1: ────────■───────────────────────────────────────────■──────────»
«            ┌─┴─┐      ┌───────────────────────┐        ┌─┴─┐        »
« q_2: ──────┤ X ├──────┤ Rz(0.608662657695409) ├────────┤ X ├────────»
«            └───┘      └───────────────────────┘        └───┘        »
« q_3: ────────────────────────────────────────────────────■──────────»
«      